In [135]:
import os
import pandas as pd
import numpy as np
from functools import reduce
from sklearn.impute import SimpleImputer
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.model_selection import train_test_split, validation_curve
from sklearn.preprocessing import PolynomialFeatures, StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LinearRegression, Lasso, Ridge

In [2]:
path = os.path.join(os.getcwd(), 'data', 'working')

In [3]:
df = pd.read_csv(os.path.join(path, 'AHS Household Combined.csv'))
varcat = pd.read_csv(os.path.join(os.getcwd(), 'data', 'concordance', 'varclass.csv'))

In [4]:
df_fh = df[df['FIRSTHOME']==1].copy()

In [72]:
vars_less_wgt = [i for i in list(df_fh.columns) if 'WGT' not in i]
vars_less_wgt = [i for i in vars_less_wgt if 'WEIGHT' not in i]
vars_less_wgt_flags = [i for i in vars_less_wgt if not i.startswith('J')]
df_fh2 = df_fh[vars_less_wgt_flags].copy()

In [73]:
df_fh2.replace(-9, np.nan, inplace=True)

In [91]:
miss_thresh = len(df_fh2) * .80
df_fh3 = df_fh2.dropna(thresh=miss_thresh, axis=1)

So begins the imputing...

In [113]:
df_varcat = pd.merge(pd.DataFrame(df_fh3.columns), varcat, how='left', 
                     left_on=pd.DataFrame(df_fh3.columns)[0], right_on=['Variable'])
target_vars = list(df_varcat['Variable'][df_varcat['Grouping'] == 'Target'])
cont_vars = list(df_varcat['Variable'][df_varcat['Grouping'] == 'Continuous'])
cat_vars = list(df_varcat['Variable'][df_varcat['Grouping'] == 'Categorical'])
binary_vars = list(df_varcat['Variable'][df_varcat['Grouping'] == 'Binary'])

In [158]:
numeric_transformer = Pipeline(steps=[('imputer', SimpleImputer(strategy='median')), 
                                      ('scaler', StandardScaler())])

In [178]:
categorical_transformer = Pipeline(steps=[('imputer', SimpleImputer(strategy='most_frequent')),
                                          ('getdummies', OneHotEncoder(handle_unknown='ignore'))])

In [179]:
binary_transformer = Pipeline(steps=[('imputer', SimpleImputer(strategy='most_frequent'))])

In [189]:
numeric_features = df_fh3[cont_vars]
categorical_features = df_fh3[cat_vars]
binary_features = df_fh3[binary_vars]

In [192]:
preprocessor = ColumnTransformer(transformers=[('num', numeric_transformer, numeric_features),
                                               ('cat', categorical_transformer, categorical_features),
                                               ('binary', binary_transformer, binary_features)])

In [182]:
y = np.array(df_fh3['RATINGHS'])
X = np.array(df_fh3.drop(['RATINGHS','RATINGNH','CONTROL','YEAR','FIRSTHOME'], axis=1))

In [183]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0, train_size=0.66)

C:\Users\Michael\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:2179: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


In [185]:
model = Pipeline(steps=[('preprocessor', preprocessor),
                        ('classifier', LinearRegression())])

In [186]:
model = LinearRegression()

In [187]:
model.fit(X_train, y_train)

ValueError: Input contains NaN, infinity or a value too large for dtype('float64').

In [ ]:
print("model score: %.3f" % clf.score(X_test, y_test))